In [0]:
from io import StringIO

In [0]:
filepath = '/FileStore/tables/hw3/userdata.txt'

In [0]:
file = sc.textFile(filepath)

In [0]:
# zip with index will create <line number, line> tokens and then we flatMap to create <lowercase token, line number> and then we go through it again so we can separate tokens with multiple words in them
inverted_index_mapped = file.zipWithIndex().flatMap(lambda x: [(i.lower(), x[1]) for i in x[0].strip().split(',')]).flatMap(lambda x: [(i, x[1]) for i in x[0].strip().split(' ')])

In [0]:
# group by key and produce <token, list of line numbers token appears in>
inverted_index = inverted_index_mapped.groupByKey().map(lambda x: (x[0], list(x[1])))

In [0]:
filtered_inverted_index = inverted_index.filter(lambda x: not any(char.isdigit() for char in x[0])).map(lambda x: (x[0], len(x[1])))

In [0]:
for i in filtered_inverted_index.take(10):
    print(i)

('evangeline', 3)
('rogers', 135)
('loveland', 12)
('us', 49995)
('robert', 885)
('cottrell', 4)
('sycamore', 183)
('circle', 664)
('texas', 3878)
('dinvis', 1)


In [0]:
# get maximum occurrences for a specific word
max_occurrences = filtered_inverted_index.values().max()
max_occurrences

Out[61]: 49995

In [0]:
token_maximum = filtered_inverted_index.filter(lambda x: x[1] == max_occurrences)

In [0]:
# outputting to file

with StringIO() as file:
        
    for i in token_maximum.collect():
        token = i[0]
        occurrences = i[1]
        
        output = f"{token}\t{occurrences}\n"
        
        file.write(output)
        
    dbutils.fs.put("/FileStore/tables/Hw3_Q2_output.txt", file.getvalue(), True)

Wrote 9 bytes.
